#### Team members:
* ...padhik2s
* ...pnutak2s
* ...

# Robot Behaviour Management Using State Machines and Behaviour Trees

In this assignment, we will compare the implementation of behaviour trees and state machines to establish primary safety features for a robot; this includes situations such as the battery level falling below a certain threshold and avoiding potential collisions. In other words, we want to implement both a behaviour tree and a state machine to achieve the same functionality.

We particularly want the robot to behave as follows:
* *The battery level falling below a given threshold*: The robot starts rotating in place until the level is above the threshold again (you can control the battery level by publishing to a topic).
* *A collision is about to happen*: The robot stops moving and needs to be moved to a safe distance manually.

You will use the Robile simulation for testing your implementation. For your submission, you need to add your code to the appropriate cells below; however, note that, to actually test your implementation, you need to integrate the code in a local ROS package and perform all tests on your local machine.

## Robot Safety Functionalities Using a State Machine [45 points]

To implement a state machine, we will use the [SMACH framework](https://wiki.ros.org/smach/Tutorials). SMACH as such is ROS independent, but `executive_smach` provides a ROS integration, which we will be using in this assignment.

Your task here is to implement the necessary states and set up the state machine to achieve the desired robot functionality.

In [1]:
### Implement the safety functionalities for the Robile by setting up
### a state machine and implementing all required states here

import rclpy
import smach

from std_msgs.msg import String
from sensor_msgs.msg import LaserScan, BatteryState
from geometry_msgs.msg import Twist
import yaml
import time

# Reference: https://wiki.ros.org/smach/Tutorials/Simple%20State%20Machine [but in ROS1]

class MonitorBatteryAndCollision(smach.State):
    """State to monitor the battery level and possible collisions
    """
    def __init__(self, node):
        # TODO: define outcomes, class variables, and desired publisher/subscribers
        super().__init__(outcomes=['safe', 'low_battery', 'collision'])
        self.node = node
        self.battery_subscriber = node.create_subscription(BatteryState,'/battery_voltage', self.battery_callback, 10)
        self.laser_subscriber = node.create_subscription(LaserScan, '/scan', self.laser_callback, 10)
        self.battery_level = 100.0
        self.collision_threshold = 0.5  # meters
        self.collision_detected = False
        
    def battery_callback(self, msg):
        self.battery_level = msg.voltage

    def laser_callback(self, msg):
        if min(msg.ranges) < self.collision_threshold:
            self.collision_detected = True
            
    def execute(self, userdata):
        # TODO: implement state execution logic and return outcome
        # YOUR CODE HERE
        if self.battery_level < 20.0:
            return 'low_battery'
        elif self.collision_detected:
            return 'collision'
        return 'safe'


class RotateBase(smach.State):
    """State to rotate the Robile base
    """
    def __init__(self, node):
        # TODO: define outcomes, class variables, and desired publisher/subscribers
        super().__init__(outcomes=['rotated', 'failed'])
        self.node = node
        self.cmd_vel_pub = self.node.create_publisher(Twist, '/cmd_vel', 10)

    def execute(self, userdata):
        # TODO: implement state execution logic and return outcome
        twist = Twist()
        twist.angular.z = 0.5
        try:
            self.cmd_vel_pub.publish(twist)
            time.sleep(2)
            twist.angular.z = 0
            self.cmd_vel_pub.publish(twist)
            return 'rotated'
        except Exception as e:
            return 'failed'


class StopMotion(smach.State):
    """State to stop the robot's motion
    """
    def __init__(self, node):
        # TODO: define outcomes, class variables, and desired publisher/subscribers
        super().__init__(outcomes=['stopped'])
        self.node = node
        self.cmd_vel_pub = self.node.create_publisher(Twist, '/cmd_vel', 10)


    def execute(self, userdata):
        # TODO: implement state execution logic and return outcome
        twist = Twist()
        self.cmd_vel_pub.publish(twist)  # Publish zero velocity to stop the robot
        return 'stopped'


# TODO: define any additional states if necessary
### YOUR CODE HERE ###

def main(args=None):
    """Main function to initialise and execute the state machine
    """

    # TODO: initialise a ROS2 node, set any threshold values, and define the state machine
    # YOUR CODE HERE
    rclpy.init(args=args)
    node = rclpy.create_node('robot_safety')

    # State machine Setup
    sm = smach.StateMachine(outcomes=['battery_low', 'collision_detected', 'all_safe'])
    with sm:
        smach.StateMachine.add('MONITOR', MonitorBatteryAndCollision(node),
                               transitions={'safe': 'ROTATE_BASE', 
                                            'low_battery': 'battery_low', 
                                            'collision': 'collision_detected'})
        smach.StateMachine.add('ROTATE_BASE', RotateBase(node),
                               transitions={'rotated': 'STOP_MOTION', 
                                            'failed': 'collision_detected'})
        smach.StateMachine.add('STOP_MOTION', StopMotion(node),
                               transitions={'stopped': 'all_safe'})

    #SMACH plan
    outcome = sm.execute()
    rclpy.spin(node)  # node run while state machine is running
    node.destroy_node()
    rclpy.shutdown()

    

if __name__ == "__main__":
    main()

[ DEBUG ] : Adding state (MONITOR, <__main__.MonitorBatteryAndCollision object at 0x70653eeff640>, {'safe': 'ROTATE_BASE', 'low_battery': 'battery_low', 'collision': 'collision_detected'})
[ DEBUG ] : Adding state 'MONITOR' to the state machine.
[ DEBUG ] : State 'MONITOR' is missing transitions: {}
[ DEBUG ] : TRANSITIONS FOR MONITOR: {'safe': 'ROTATE_BASE', 'low_battery': 'battery_low', 'collision': 'collision_detected'}
[ DEBUG ] : Adding state (ROTATE_BASE, <__main__.RotateBase object at 0x70653eeff610>, {'rotated': 'STOP_MOTION', 'failed': 'collision_detected'})
[ DEBUG ] : Adding state 'ROTATE_BASE' to the state machine.
[ DEBUG ] : State 'ROTATE_BASE' is missing transitions: {}
[ DEBUG ] : TRANSITIONS FOR ROTATE_BASE: {'rotated': 'STOP_MOTION', 'failed': 'collision_detected'}
[ DEBUG ] : Adding state (STOP_MOTION, <__main__.StopMotion object at 0x70653eefc0a0>, {'stopped': 'all_safe'})
[ DEBUG ] : Adding state 'STOP_MOTION' to the state machine.
[ DEBUG ] : State 'STOP_MOTION' i

KeyboardInterrupt: 

## Robot Safety Functionalities Using a Behaviour Tree [45 points]

The majority of implementations of behaviour trees in robotics are using `BehaviorTree.CPP` in cpp and [py_trees](https://py-trees.readthedocs.io/en/devel/) in Python. [py_trees_ros](https://py-trees-ros-tutorials.readthedocs.io/en/release-2.1.x/tutorials.html) is a wrapper for `py_trees` to integrate it with ROS, which we will use in this assignment.

Your task here is to implement the necessary behaviours and set up the behaviour tree to achieve the desired robot functionality.

Implement the required behaviours in the cell below. [35 points]

In [ ]:
### Implement the safety functionalities for the Robile by implementing all
### required behaviours here. Feel free to define additional behaviours if necessary

import rclpy
import py_trees as pt
import py_trees_ros as ptr
from geometry_msgs.msg import Twist
from std_msgs.msg import Float32
from sensor_msgs.msg import LaserScan
from rclpy.qos import QoSProfile, QoSReliabilityPolicy, QoSHistoryPolicy

class Rotate(pt.behaviour.Behaviour):
    """Rotates the robot about the z-axis 
    """
    def __init__(self, name="rotate platform",
                 topic_name="/cmd_vel",
                 ang_vel=1.0):
        super(Rotate, self).__init__(name)

        # TODO: initialise any necessary class variables
        # YOUR CODE HERE
#         raise NotImplementedError()
        self.topic_name = topic_name
        self.ang_vel = ang_vel
        self.publisher = None


    def setup(self, **kwargs):
        """Setting up things which generally might require time to prevent delay in the tree initialisation
        """
        self.logger.info("[ROTATE] setting up rotate behaviour")
        try:
            self.node = kwargs['node']
            self.publisher = self.node.create_publisher(Twist, self.topic_name, 10)
        except KeyError as e:
            error_message = "didn't find 'node' in setup's kwargs [{}][{}]".format(self.qualified_name)
            raise KeyError(error_message) from e 
        return True

    def update(self):
        """Rotates the robot at the maximum allowed angular velocity.
        Note: The actual behaviour is implemented here.

        """
        self.logger.info("[ROTATE] update: updating rotate behaviour")
        self.logger.debug("%s.update()" % self.__class__.__name__)

        # TODO: implement the primary function of the behaviour and decide which status to return 
        # based on the structure of your behaviour tree

        # Hint: to return a status, for example, SUCCESS, pt.common.Status.SUCCESS can be used

        # YOUR CODE HERE
        twist = Twist()
        twist.angular.z = self.ang_vel
        self.publisher.publish(twist)
        return pt.common.Status.RUNNING 


    def terminate(self, new_status):
        """Trigerred once the execution of the behaviour finishes, 
        i.e. when the status changes from RUNNING to SUCCESS or FAILURE
        """
        self.logger.info("[ROTATE] terminate: publishing zero angular velocity")

        # TODO: implement the termination of the behaviour, i.e. what should happen when the behaviour 
        # finishes its execution

        twist = Twist()
        twist.angular.z = 0.0
        self.publisher.publish(twist)
        super().terminate(new_status)

class StopMotion2bb(pt.behaviour.Behaviour):
    """Stops the robot when it is controlled using a joystick or with a cmd_vel command
    """
    
    # TODO: Implement a behaviour to stop the robot's motion

    # YOUR CODE HERE
    def __init__(self, name="Stop Motion", topic_name="/cmd_vel"):
        super(StopMotion2bb, self).__init__(name)
        self.topic_name = topic_name
        self.publisher = None
        self.initialized = False

    def update(self):
        if not self.initialized:
            try:
                self.publisher = self.node.create_publisher(Twist, self.topic_name, 10)
                self.initialized = True
            except AttributeError as e:
                self.logger.error(error_message)
                raise AttributeError(error_message) from e

        twist = Twist()
        self.publisher.publish(twist)
        return pt.common.Status.SUCCESS


class BatteryStatus2bb(ptr.subscribers.ToBlackboard):
    """Checks the battery status
    """
    def __init__(self, battery_voltage_topic_name: str="/battery_voltage",
                 name: str='Battery2BB',
                 threshold: float=30.0):
        super().__init__(name=name,
                         topic_name=battery_voltage_topic_name,
                         topic_type=Float32,
                         blackboard_variables={'battery': 'data'},
                         initialise_variables={'battery': 100.0},
                         clearing_policy=pt.common.ClearingPolicy.NEVER,  # to decide when data should be cleared/reset.
                         qos_profile=ptr.utilities.qos_profile_unlatched())
        self.blackboard.register_key(key='battery_low_warning', access=pt.common.Access.WRITE)

        # YOUR CODE HERE
#         raise NotImplementedError()
        self.threshold = threshold


    def update(self):
        """Calls the parent to write the raw data to the blackboard and then check against the
        threshold to determine if a low warning flag should also be updated.
        """
        self.logger.info('[BATTERY] update: running battery_status2bb update')
        self.logger.debug("%s.update()" % self.__class__.__name__)
        
        """check battery voltage level stored in self.blackboard.battery. By comparing with 
        threshold value, update the value of self.blackboad.battery_low_warning
        """

        # TODO: based on the battery voltage level, update the value of self.blackboard.battery_low_warning
        # and return the status of the behaviour based on your logic of the behaviour tree

        super().update()
        voltage = self.blackboard.battery
        if voltage < self.threshold:
            self.blackboard.battery_low_warning = True
            return pt.common.Status.FAILURE
        else:
            self.blackboard.battery_low_warning = False
            return pt.common.Status.SUCCESS


class LaserScan2bb(ptr.subscribers.ToBlackboard):
    """Checks the laser scan measurements to avoid possible collisions.
    """
    def __init__(self, topic_name: str="/scan",
                 name: str='Scan2BB',
                 safe_range: float=0.25):
        super().__init__(name=name,
                         topic_name=topic_name,
                         topic_type=LaserScan,
                         blackboard_variables={'laser_scan':'ranges'},
                         clearing_policy=pt.common.ClearingPolicy.NEVER,  # to decide when data should be cleared/reset.
                         qos_profile=QoSProfile(reliability=QoSReliabilityPolicy.RMW_QOS_POLICY_RELIABILITY_BEST_EFFORT,
                                                history=QoSHistoryPolicy.RMW_QOS_POLICY_HISTORY_KEEP_LAST,
                                                depth=10))
        
        # TODO: initialise class variables and blackboard variables
        self.safe_range = safe_range

    def update(self):
        # TODO: impletment the update function to check the laser scan data and update the blackboard variable
        super().update()
        ranges = self.blackboard.laser_scan
        if any(r < self.safe_range for r in ranges):
            return pt.common.Status.FAILURE  # Indicates a collision risk
        return pt.common.Status.SUCCESS

Now, set up and initialise your behaviour tree in the cell below. [10 points]

In [ ]:
### Implement a behaviour tree using your previously implemented behaviours here

import py_trees as pt
import py_trees_ros as ptr
import operator

import py_trees.console as console
import rclpy
import sys

def create_root() -> pt.behaviour.Behaviour:
    """Structures a behaviour tree to monitor the battery status, and start
    to rotate if the battery is low and stop if it detects an obstacle in front of it.
    """

    # we define the root node
    root = pt.composites.Parallel(name="root",
                                  policy=pt.common.ParallelPolicy.SuccessOnAll(synchronise=False))    

    ### we create a sequence node called "Topics2BB" and a selector node called "Priorities"
    topics2BB = pt.composites.Sequence("Topics2BB", memory=False)
    priorities = pt.composites.Selector("Priorities", memory=False)

    ### we create an "Idle" node, which is a running node to keep the robot idle
    idle = pt.behaviours.Running(name="Idle")
    
    """
    TODO:  The first and second level of the tree structure is defined above, but please
    define the rest of the tree structure.

    Class definitions for your behaviours are provided in behaviours.py; you also need to fill out
    the behaviour implementations!

    HINT: Some behaviours from pt.behaviours may be useful to use as well.
    """

    #sstart behaviours
    battery_monitor = BatteryStatus2bb(topic_name="/battery_voltage")
    scan_monitor = LaserScan2bb(topic_name="/scan", safe_range=0.2)
    rotate = Rotate(name="Rotate", topic_name="/cmd_vel", ang_vel=0.5)
    stop_motion = StopMotion2bb(name="Stop Motion", topic_name="/cmd_vel")

    #Topics2BB: Monitor the battery and the laser scan
    topics2BB.add_children([battery_monitor, scan_monitor])

    #priority selector
    # Check battery first, if low ->rotate, if collision detected ->stop
    check_battery_low = pt.decorators.Condition(
        child=battery_monitor,
        name="Low Battery?",
        condition=operator.attrgetter("battery_low_warning"),
        value=True
    )

    detect_obstacle = pt.decorators.Condition(
        child=scan_monitor,
        name="Obstacle Near?",
        condition=operator.attrgetter("collision_detected"),
        value=True
    )

    stop_if_needed = pt.composites.Selector(name="Stop if Needed")
    stop_if_needed.add_children([detect_obstacle, stop_motion])

    rotate_if_safe = pt.composites.Sequence(name="Rotate if Safe")
    rotate_if_safe.add_children([pt.decorators.Inverter(detect_obstacle), rotate])

    priorities.add_children([stop_if_needed, check_battery_low, rotate_if_safe, idle])


    # TODO: construct the behaviour tree structure using the nodes and behaviours defined above
    # HINT: for reference, the sample tree structure in the README.md file might be useful

    root.add_children([topics2BB, priorities])

    # YOUR CODE HERE
#     raise NotImplementedError()

    return root

def main():
    """Initialises and executes the behaviour tree
    """
    rclpy.init(args=None)

    root = create_root()
    tree = ptr.trees.BehaviourTree(root=root, unicode_tree_debug=True)

    try:
        tree.setup(timeout=30.0)
    except ptr.exceptions.TimedOutError as e:
        console.logerror(console.red + "failed to setup the tree, aborting [{}]".format(str(e)) + console.reset)
        tree.shutdown()
        rclpy.try_shutdown()
        sys.exit(1)
    except KeyboardInterrupt:
        # user-initiated shutdown
        console.logerror("tree setup interrupted")
        tree.shutdown()
        rclpy.try_shutdown()
        sys.exit(1)
    
    # frequency of ticks
    tree.tick_tock(period_ms=100)    

    try:
        rclpy.spin(tree.node)
    except (KeyboardInterrupt, rclpy.executors.ExternalShutdownException):
        pass
    finally:
        tree.shutdown()
        rclpy.try_shutdown()

if __name__ == '__main__':
    main()

## Setting up Your System for Testing

1. Please setup Ubuntu 22.04, ROS2 humble, and the Robile simulation (if not already done) by following the [documentation](https://robile-amr.readthedocs.io/en/humble/).

2. Clone the `executive_smach` repository in the src folder of your workspace, and also install the `py-trees-ros` package:
```
cd ~/ros2_ws/src/  
git clone -b ros2 https://github.com/ros/executive_smach.git  
sudo apt-get install ros-humble-py-trees ros-humble-py-trees-ros ros-humble-py-trees-ros-interfaces xcb
```

From the workspace directory, build the workspace:
```
cd ~/ros2_ws/
colcon build --symlink-install
```

Now source the workspace setup file:
```
source install/setup.bash
```
3. Create a new ROS2 python package and integrate your implementation in it

## Testing Instructions

Use the following steps to test your implementation:
- Run the Robile in simulation
- After integrating your implementation in your local ROS workspace, launch your components to test the functionalities. Note that you need to test the state machine and behaviour tree implementations independently to verify that the robot behaves exactly the same in both cases.

**In your submission, please include screenshots to verify your implementation.**

Note that, as the battery percentage is not readily available in simulation, please publish the battery percentage values manually. For instance, if the topic `/battery_voltage` is used for monitoring the battery status, you should be able to publish a battery percentage value of your choice to verify your implementation, e.g.:
```  
ros2 topic  pub /battery_voltage std_msgs/msg/Float32  data:\ 50.0\ 
```

Finally, behaviour tree visualization is not released on ROS2 humble yet, but the changes in the behaviour tree can be monitored by running the following command, which is helpful for debugging:
```
py-trees-tree-watcher
```

The following is a sample visualisation when the robot is about to collide:

![collision avoidance BT](figures/BT_watcher.png)

For getting a better intuition of behaviour trees, another sample visualisation of a similar task, but with a slightly different structure can be found below:

![collison and battery low](figures/collision_battery.png)

**Discuss any observations from your tests and include screenshots that verify your implementation in the cell below. [10 points]**

The behavior-tree allowed robot to react more flexibly and continuously check for changes like battery level or obstacles, adjusting quickly. This made the robot feel smoother and more responsive. while the state machine with SMACH handled tasks in a more step-by-step manner, which made the robot's reactions feel a bit slower and less fluid when conditions changed rapidly.
So BT is adaptive and State Machines are bit rigid (one task at a time)

### Battery Checkpoint
![Battery Checkpoint](figures/Battery_checkpoint1.png)

### Behavior Tree Idle
![Behavior Tree Collision](figures/BT_WithoutCollision.png)


### Behavior Tree Collision
![Behavior Tree Collision](figures/BT_Collision.png)